In [1]:
import joblib
import pickle

In [2]:
from tensorflow.keras.models import load_model
import cv2
import mediapipe as mp
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [3]:
from flask_socketio import SocketIO, emit

In [4]:
import os

In [5]:
from flask import Flask, render_template, Response, jsonify

In [16]:

app = Flask(__name__)
socketio = SocketIO(app)
# Mediapipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2)
mp_drawing = mp.solutions.drawing_utils

'''
def extract_hand_landmarks(image):
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0].landmark
        landmarks_array = [[landmark.x, landmark.y, landmark.z] for landmark in landmarks]
        return landmarks_array
    return None'''
last_landmarks = None
last_bounding_box = None
last_hand_label = None
def extract_hand_landmarks(image):
    global last_landmarks, last_bounding_box, last_hand_label
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    if results.multi_hand_landmarks:
        for index, landmarks in enumerate(results.multi_hand_landmarks):
            # Draw landmarks
            mp_drawing.draw_landmarks(image, landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Update bounding box
            lmx = [landmark.x for landmark in landmarks.landmark]
            lmy = [landmark.y for landmark in landmarks.landmark]
            min_x, max_x, min_y, max_y = int(min(lmx)*image.shape[1]), int(max(lmx)*image.shape[1]), int(min(lmy)*image.shape[0]), int(max(lmy)*image.shape[0])
            cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)
            
            # Update hand label
            hand_label = results.multi_handedness[index].classification[0].label
            hand_label = "Right" if hand_label == "Left" else "Left"
            cv2.putText(image, hand_label, (min_x, min_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            
            # Store current landmarks, bounding box and label for future frames
            last_landmarks = landmarks
            last_bounding_box = (min_x, min_y, max_x, max_y)
            last_hand_label = hand_label
        landmarks = results.multi_hand_landmarks[0].landmark
        landmarks_array = [[landmark.x, landmark.y, landmark.z] for landmark in landmarks]
        return landmarks_array
    else:
        # If no hand is detected, use the last detected landmarks, bounding box, and label
        if last_landmarks and last_bounding_box:
            mp_drawing.draw_landmarks(image, last_landmarks, mp_hands.HAND_CONNECTIONS)
            min_x, min_y, max_x, max_y = last_bounding_box
            cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)
            if last_hand_label:
                cv2.putText(image, last_hand_label, (min_x, min_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return None

def landmarks_to_heatmap(landmarks, img_size=(128, 128), blob_size=5):
    heatmap = np.zeros(img_size)
    for landmark in landmarks:
        x, y = int(landmark[0] * img_size[0]), int(landmark[1] * img_size[1])
        cv2.circle(heatmap, (x, y), blob_size, (255, 255, 255), -1)
    return heatmap

def preprocess_image(img, img_size=(128, 128)):
    landmarks = extract_hand_landmarks(img)
    if landmarks:
        heatmap = landmarks_to_heatmap(landmarks, img_size)
        return heatmap / 255.0
    return None

feature_extractor = load_model("cnn_model_3_mediapipe_90.h5")
dt_classifier = joblib.load('dt_classifier_3_mediapipe_90.pkl')
unique_labels = joblib.load('unique_labels_3_mediapipe_90.pkl')
img_size = (128, 128)
path = "D:/B Tech/IV Year/project/project new/dataset_mediapipe"
#def generate():

@socketio.on('start_gesture_recognition')
def get_gesture():
    # Your gesture recognition logic here
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
       # resized_frame = cv2.resize(frame, (int(frame.shape[1]*1.5), int(frame.shape[0]*1.5)))
        processed_img = preprocess_image(frame, img_size)
    
        if processed_img is not None:
            processed_img_reshaped = processed_img.reshape((-1, img_size[0], img_size[1], 1))
            features = feature_extractor.predict(processed_img_reshaped)
            prediction = dt_classifier.predict(features)
            predicted_gesture = unique_labels[prediction[0]]
            final_output = os.path.basename(predicted_gesture)
            text_position = (int(frame.shape[1] * 0.05), int(frame.shape[0] * 0.05))
            cv2.putText(frame, final_output, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)
            socketio.emit('gesture_update', final_output)
        _, jpeg = cv2.imencode('.jpg', frame)
        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + jpeg.tobytes() + b'\r\n\r\n')

        #emit('gesture_update', (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + jpeg.tobytes() + b'\r\n\r\n').decode('utf-8'))
       # socketio.emit('frame_update', jpeg.tobytes().decode('base64'))
        


@app.route('/video_feed')
def video_feed():
    return Response(get_gesture(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def index():
    return render_template('index2.html')

if __name__ == "__main__":
    socketio.run(app, allow_unsafe_werkzeug=True)


Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Oct/2023 12:06:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2023 12:06:51] "GET /static/icon0.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:06:52] "GET /static/js/your_script.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:06:52] "GET /static/icon1.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:06:52] "GET /static/icon2.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:06:52] "GET /static/icon3.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:06:52] "GET /static/menu2.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:06:52] "GET /static/background.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:06:52] "GET /static/icon4.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:06:52] "GET /socket.io/?EIO=4&transport=polling&t=Oj6fhL3 HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2023 12:06:52] "POST /socket.io/?EIO=4&transport=polling&t=Oj6fhNS&sid=R8THpsnRh064_WAdAAAA HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2023 1

1/1 [==============================] - 0s 119ms/step


127.0.0.1 - - [19/Oct/2023 12:08:00] "GET /static/quantity1.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:08:00] "GET /static/quantity2.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:08:00] "GET /static/quantity3.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:08:00] "GET /static/quantity5.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:08:00] "GET /static/quantity4.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:08:13] "GET /socket.io/?EIO=4&transport=websocket&sid=R8THpsnRh064_WAdAAAA HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2023 12:08:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2023 12:08:41] "GET /static/icon0.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:08:42] "GET /static/icon1.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:08:42] "GET /static/icon2.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:08:42] "GET /static/js/your_script.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:08:42] "GET /static/icon3.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Oct/2023 12:0